# Self-Reflective Retrieval-Augmented Generation (Self-RAG) with LangGraph

## Project Goal
The goal of this project is to experiment with Self-RAG with LangGraph.

This project is based on ["Self-Reflective RAG with LangGraph"](https://blog.langchain.dev/agentic-rag-with-langgraph/) blog by LangChain Team.

## Self-RAG Introduction

Large Language Models (LLMs), can generate impressive text but often produce **factually incorrect responses** because they rely on **pre-trained knowledge** and don’t dynamically verify facts. To solve this, researchers have developed **Self-Reflective Retrieval-Augmented Generation (Self-RAG)**—a framework that enables LLMs to fetch relevant external information **when needed** and **self-assess** their own outputs for accuracy and reliability.

## How Self-RAG Works

Self-RAG improves the traditional Retrieval-Augmented Generation (RAG) process by introducing three core components:



**1.  Adaptive Retrieval:**

* Unlike standard retrieval models, which fetch a fixed number of documents, Self-RAG allows the model to determine when and what information to retrieve based on the complexity of the question.
* This helps the model avoid unnecessary information overload and focus only on relevant data.



**2.  Response Generation:**

* After retrieving information, the LLM integrates it into a coherent and accurate response.

**3.  Self-Reflection with Specialized Tokens:**

* Self-RAG uses reflection tokens that allow the model to evaluate its own response in real time.
* These special tokens are embedded in the model’s reasoning process to check and refine the accuracy of its generated responses.

## Self-Reflection Tokens

Self-RAG introduces four key reflection tokens, which play a crucial role in the model’s self-reflection and response evaluation:

**1. ISREL (Information Relevance Token)**
* This token is used to evaluate whether the retrieved information is relevant to the question being asked.
* The model asks itself: “Is this document useful for answering the query?”

**2. ISSUP (Information Support Token)**
* This token helps determine whether the retrieved information supports the generated response.
* The model assesses: “Does the information I retrieved actually back up my answer?”
* This step ensures that the response is not based on assumptions but on factual sources.

**3. ISUSE (Information Usefulness Token)**
* This token checks whether the retrieved information was actually used in the response.
* The model verifies: “Did I properly incorporate this information into my answer?”
* It prevents the model from retrieving documents but then ignoring them in the final response.

**4.	ISERR (Information Error Token)**
* This token allows the model to detect potential mistakes in its generated response.
* The model asks: “Did I make an error in my answer?”
* If an error is detected, the model can rephrase, correct, or retrieve more information to improve accuracy.

## References

1. [“Self-RAG: Learning to Retrieve, Generate, and Critique through Self-Reflection" ](https://arxiv.org/abs/2310.11511)by Akari Asai, Zeqiu Wu, Yizhong Wang, Avirup Sil, Hannaneh Hajishirzi. 2023

2. ["Self-Reflective RAG with LangGraph"](https://blog.langchain.dev/agentic-rag-with-langgraph/) blog is available at LangChain.




## Setup

In [ ]:
# Mounting to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd "your-path-here"

In [ ]:
%%capture
! pip install langgraph langchain faiss-cpu langchain-openai langchain_community chromadb
!pip install -U langchain langgraph langchain_openai langchain-community

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "your-OpenAI-API-key-here"

## Prepare Data

We will use articles about Quantum Computing in this project:
* [Amazon Unveils Its First Quantum Computing Chip](https://www.wsj.com/articles/amazon-unveils-its-first-quantum-computing-chip-750ee5da)
* [Google’s New Quantum Computer Chip](https://www.businessinsider.com/google-unveiled-quantum-computer-chip-willow-2024-12)
* [The Age of Quantum Software Has Already Started](https://www.wsj.com/articles/the-age-of-quantum-software-has-already-started-854eccfa)

In [ ]:
urls = [
    "https://www.wsj.com/articles/amazon-unveils-its-first-quantum-computing-chip-750ee5da",
    "https://www.businessinsider.com/google-unveiled-quantum-computer-chip-willow-2024-12",
    "https://www.wsj.com/articles/the-age-of-quantum-software-has-already-started-854eccfa"
]


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

## Create Embeddings and Vector Store

In [ ]:
# Generate embeddings for the documents and store them in a vector store for efficient retrieval
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=OpenAIEmbeddings(),
)

## Set Up the Retriever

In [ ]:
# Set up a retriever from the vector store to fetch relevant documents based on queries
retriever = vectorstore.as_retriever()

## LLMs

In [ ]:
### Retrieval Grader

from langchain_core.prompts import ChatPromptTemplate
#from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field


# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )


# LLM with function call
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Prompt
system = """You are a grader assessing relevance of a retrieved document to a user question. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader
question = "Please explain shortly Amazon's first quantum computing chip features"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

/usr/local/lib/python3.11/dist-packages/langchain_openai/chat_models/base.py:1390: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


binary_score='yes'


In [ ]:
### Generate

from langchain import hub
from langchain_core.output_parsers import StrOutputParser
import textwrap

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
generation = rag_chain.invoke({"context": docs, "question": question})

# Print question and response
print("\nQuestion: %s" % question)
print("-" * 80)

# Print formatted final answer
wrapped_answer = textwrap.fill(generation, width=80)  # Wrap text for readability
print("\nAnswer:\n" + wrapped_answer)

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(



Question: Please explain shortly Amazon's first quantum computing chip features
--------------------------------------------------

Answer:
Amazon's first quantum computing chip, Ocelot, can reduce quantum computing
errors by up to 90%, leading to more reliable quantum computers. The chip is a
significant step in the development of useful and cost-effective quantum
computing technology. Amazon's cloud-computing business claims that Ocelot can
lower the costs associated with reducing quantum computing errors.


In [ ]:
question = "Please describe the main features of Google’s New Quantum Computer Chip"
generation = rag_chain.invoke({"context": docs, "question": question})

# Print question and response
print("\nQuestion: %s" % question)
print("-" * 80)

# Print formatted final answer
wrapped_answer = textwrap.fill(generation, width=80)  # Wrap text for readability
print("\nAnswer:\n" + wrapped_answer)


Question: Please describe the main features of Google’s New Quantum Computer Chip
--------------------------------------------------

Answer:
Google's new quantum computer chip, called Willow, reduces errors and
outperforms standard benchmarks. The chip can perform a standard benchmark
computation in under five minutes, a task that would take current supercomputers
10 septillion years. This development represents a key milestone in the race to
build accurate quantum computers with practical applications.


In [ ]:
question = "Please compare Google’s new quantum computer chip with Amazon's first quantum computing chip"
generation = rag_chain.invoke({"context": docs, "question": question})

# Print question and response
print("\nQuestion: %s" % question)
print("-" * 80)

# Print formatted final answer
wrapped_answer = textwrap.fill(generation, width=80)  # Wrap text for readability
print("\nAnswer:\n" + wrapped_answer)


Question: Please compare Google’s new quantum computer chip with Amazon's first quantum computing chip
--------------------------------------------------------------------------------

Answer:
Google's new quantum computer chip, Willow, vastly outperforms standard
benchmarks and reduces errors significantly. Amazon's first quantum computing
chip, Ocelot, is claimed to lower the costs of reducing quantum computing errors
by up to 90%. Both chips represent significant advancements in quantum computing
technology.


## Hallucination Grader

In [ ]:
# Data model
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""

    binary_score: str = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )


# LLM with function call
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeHallucinations)

# Prompt
system = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n
     Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."""
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
    ]
)

hallucination_grader = hallucination_prompt | structured_llm_grader
hallucination_grader.invoke({"documents": docs, "generation": generation})

/usr/local/lib/python3.11/dist-packages/langchain_openai/chat_models/base.py:1390: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


GradeHallucinations(binary_score='yes')

## Answer Grader

In [ ]:
# Data model
class GradeAnswer(BaseModel):
    """Binary score to assess answer addresses question."""

    binary_score: str = Field(
        description="Answer addresses the question, 'yes' or 'no'"
    )


# LLM with function call
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeAnswer)

# Prompt
system = """You are a grader assessing whether an answer addresses / resolves a question \n
     Give a binary score 'yes' or 'no'. Yes' means that the answer resolves the question."""
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
    ]
)

answer_grader = answer_prompt | structured_llm_grader
answer_grader.invoke({"question": question, "generation": generation})

/usr/local/lib/python3.11/dist-packages/langchain_openai/chat_models/base.py:1390: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


GradeAnswer(binary_score='yes')

## Question Re-writer

In [ ]:
# LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Prompt
system = """You are question re-writer that converts an input question to a better version that is optimized \n
     for vectorstore retrieval. Look at the input and try to reason about the underlying semantic intent / meaning."""
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Here is the initial question: \n\n {question} \n Formulate an improved question.",
        ),
    ]
)

question_rewriter = re_write_prompt | llm | StrOutputParser()
question_rewriter.invoke({"question": question})

"Compare the features and performance of Google's latest quantum computer chip with Amazon's initial quantum computing chip."

## Graph

In [ ]:
# Graph State
from typing import List
from typing_extensions import TypedDict


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
    """

    question: str
    generation: str
    documents: List[str]

In [ ]:
### Nodes

def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.get_relevant_documents(question)
    return {"documents": documents, "question": question}


def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}


def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue
    return {"documents": filtered_docs, "question": question}


def transform_query(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    question = state["question"]
    documents = state["documents"]

    # Re-write question
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question}


### Edges


def decide_to_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    state["question"]
    filtered_documents = state["documents"]

    if not filtered_documents:
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print(
            "---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---"
        )
        return "transform_query"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"


def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = score.binary_score

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question, "generation": generation})
        grade = score.binary_score
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        pprint("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

## Build Graph

In [ ]:
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generatae
workflow.add_node("transform_query", transform_query)  # transform_query

# Build graph
workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "generate": "generate",
    },
)
workflow.add_edge("transform_query", "retrieve")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "transform_query",
    },
)

# Compile
app = workflow.compile()

In [ ]:
# Run
inputs = {"question": "What are the main features of Amazon's first quantum computing chip?"}

for output in app.stream(inputs):
    for key, value in output.items():
        # Print the node name
        print(f"\n Node '{key}':")

        # Optional: Print full state at each node (formatted for readability)
        if "keys" in value:
            formatted_text = textwrap.fill(str(value["keys"]), width=80, subsequent_indent="    ")
            print(f"   State: {formatted_text}")

    print("\n" + "-" * 80 + "\n")  # Separator for clarity

# Final generation output with text wrapping
if "generation" in value:
    print("\n Final Generated Response:\n")
    print(textwrap.fill(value["generation"], width=80, subsequent_indent=""))

---RETRIEVE---

 Node 'retrieve':

--------------------------------------------------------------------------------

---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---

 Node 'grade_documents':

--------------------------------------------------------------------------------

---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---

 Node 'generate':

--------------------------------------------------------------------------------


 Final Generated Response:

Amazon's first quantum computing chip is called Ocelot. It is designed to reduce
quantum computing errors by up to 90%, making quantum computers more reliable.
This chip is a significant step in the development of practical quantum
comput

In [ ]:
# Run
inputs = {"question": "What are the main features of Google’s new quantum computer chip?"}

for output in app.stream(inputs):
    for key, value in output.items():
        # Print the node name
        print(f"\n Node '{key}':")

        # Optional: Print full state at each node (formatted for readability)
        if "keys" in value:
            formatted_text = textwrap.fill(str(value["keys"]), width=80, subsequent_indent="    ")
            print(f"   State: {formatted_text}")

    print("\n" + "-" * 80 + "\n")  # Separator for clarity

# Final generation output with text wrapping
if "generation" in value:
    print("\n Final Generated Response:\n")
    print(textwrap.fill(value["generation"], width=80, subsequent_indent=""))

---RETRIEVE---

 Node 'retrieve':

--------------------------------------------------------------------------------

---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---

 Node 'grade_documents':

--------------------------------------------------------------------------------

---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---

 Node 'generate':

--------------------------------------------------------------------------------


 Final Generated Response:

Google's new quantum computer chip, Willow, reduces errors and vastly
outperforms standard benchmarks. The chip can perform a standard benchmark
computation in under five minutes, a task that would take current supercomputers
10 septillion years.

In [ ]:
# Run
inputs = {"question": "Please compare Google’s New Quantum Computer Chip with Amazon's first quantum computing chip"}

for output in app.stream(inputs):
    for key, value in output.items():
        # Print the node name
        print(f"\n Node '{key}':")

        # Optional: Print full state at each node (formatted for readability)
        if "keys" in value:
            formatted_text = textwrap.fill(str(value["keys"]), width=80, subsequent_indent="    ")
            print(f"   State: {formatted_text}")

    print("\n" + "-" * 80 + "\n")  # Separator for clarity

# Final generation output with text wrapping
if "generation" in value:
    print("\n Final Generated Response:\n")
    print(textwrap.fill(value["generation"], width=80, subsequent_indent=""))

---RETRIEVE---

 Node 'retrieve':

--------------------------------------------------------------------------------

---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---

 Node 'grade_documents':

--------------------------------------------------------------------------------

---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---

 Node 'generate':

--------------------------------------------------------------------------------


 Final Generated Response:

Google's new quantum computer chip, Willow, reduces errors and vastly
outperforms standard benchmarks, cracking a 30-year challenge in the field.
Amazon's first quantum computing chip, Ocelot, is claimed to be an important
step in the develop